**Import needed libraries**

In [66]:
import pandas as pd
import psycopg2
import configparser

In [67]:
# Acessing Credentials
config = configparser.ConfigParser()
config.read('../credentials/credentials.ini')
USERNAME = config['ElephantSql Road Lane Detection Instance']['USERNAME']
PASSWORD = config['ElephantSql Road Lane Detection Instance']['PASSWORD']
DATABASE = config['ElephantSql Road Lane Detection Instance']['DATABASE']
HOST = config['ElephantSql Road Lane Detection Instance']['HOST']

### Creating and loading road data dataframe into postgresql database

In [68]:
df = pd.DataFrame({'scene': [
                   'scene_1.mp4',
                   'scene_2.mp4',
                   'scene_3.mp4',
                   'scene_4.mp4',
                   'scene_5.mp4',
                   'scene_6.mp4',
                   'scene_7.mp4',
                   'scene_8.mp4',
                   'scene_9.mp4',
                   'scene_10.mp4',
                   'scene_11.mp4',
                   'scene_12.mp4',
                   'scene_13.mp4',
                   'scene_14.mp4',
                   'scene_15.mp4',
                   'scene_16.mp4',
                   'scene_17.mp4',
                   'scene_18.mp4'
                   ],
                   'weather': [
                   'clear',
                   'clear',
                   'clear',
                   'clear',
                   'clear',
                   'clear',
                   'clear',
                   'clear',
                   'clear',
                   'clear',
                   'clear',
                   'clear',
                   'clear',
                   'clear',
                   'clear',
                   'clear',
                   'clear',
                   'clear'
                   ],
                   'time_of_day': [
                   'afternoon',
                   'afternoon',
                   'afternoon',
                   'afternoon',
                   'afternoon',
                   'afternoon',
                   'afternoon',
                   'afternoon',
                   'afternoon',
                   'afternoon',
                   'afternoon',
                   'afternoon',
                   'afternoon',
                   'afternoon',
                   'afternoon',
                   'afternoon',
                   'afternoon',
                   'afternoon'
                   ],
                   'shadows': [
                   'false',
                   'false',
                   'false',
                   'false',
                   'false',
                   'false',
                   'true',
                   'true',
                   'true',
                   'true',
                   'true',
                   'true',
                   'true',
                   'true',
                   'true',
                   'true',
                   'true',
                   'true'
                   ],
                   'road_type': [
                   'highway',
                   'highway',
                   'highway',
                   'highway',
                   'highway',
                   'highway',
                   'highway',
                   'street',
                   'highway',
                   'highway',
                   'highway',
                   'highway',
                   'street',
                   'street',
                   'street',
                   'street',
                   'street',
                   'street'
                   ],
                   'traffic': [                   
                   'false',
                   'false',
                   'false',
                   'false',
                   'false',
                   'false',
                   'false',
                   'true',
                   'true',
                   'true',
                   'true',
                   'true',
                   'true',
                   'true',
                   'true',
                   'true',
                   'true',
                   'true'
                   ],
                   'line_condition': [
                   'smooth',
                   'smooth',
                   'smooth',
                   'smooth',
                   'smooth',
                   'smooth',
                   'smooth',
                   'mixed',
                   'poor',
                   'poor',
                   'dotted',
                   'dotted',
                   'dotted',
                   'dotted',
                   'dotted',
                   'dotted',
                   'dotted',
                   'dotted'
                   ],
                   'lane_count': [
                   '1',
                   '1',
                   '1',
                   '1',
                   '1',
                   '1',
                   '1',
                   '3',
                   '3',
                   '2',
                   '3',
                   '3',
                   '3',
                   '3',
                   '3',
                   '3',
                   '3',
                   '3'
                   ],
                   'pavement': [
                   'smooth',
                   'smooth',
                   'smooth',
                   'smooth',
                   'smooth',
                   'smooth',
                   'smooth',
                   'smooth',
                   'patchy',
                   'mixed',
                   'smooth',
                   'smooth',
                   'smooth',
                   'mixed',
                   'mixed',
                   'mixed',
                   'mixed',
                   'mixed'
                   ],
                   'detection_preformence': [
                   'good',
                   'good',
                   'good',
                   'good',
                   'good',
                   'good',
                   'good',
                   'bad',
                   'bad',
                   'bad',
                   'bad',
                   'bad',
                   'bad',
                   'bad',
                   'bad',
                   'bad',
                   'bad',
                   'bad'
                   ]
                   })

In [69]:
def connect(DATABASE, USERNAME, PASSWORD, HOST):
    """ Connect to the PostgreSQL database server """
    elephantsql_client = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')

        # Connect to ElephantSQL-hosted PostgreSQL
        elephantsql_client = psycopg2.connect(dbname=DATABASE, user=USERNAME, password=PASSWORD, host=HOST)

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    return elephantsql_client

def create_table(elephantsql_client):
    ''' Creating a table based on data found in the mr beast csv file'''

    # For the purposes of this I will handwrite the sql commands but will use sqlalchemy in future projects
    command = '''
        CREATE TABLE IF NOT EXISTS road_data_table (id                      SERIAL PRIMARY KEY,
                                                    scene                    varchar(20),
                                                    weather                 varchar(20),
                                                    time_of_day             varchar(20),
                                                    total_revenue           varchar(20),
                                                    shadows                 varchar(20),
                                                    road_type               varchar(20),
                                                    traffic                 varchar(20),
                                                    line_condition          varchar(20),
                                                    lane_count              varchar(20),
                                                    pavement                varchar(20),
                                                    detection_preformence   varchar(20))
        '''
    
    try:

        # A "cursor", a structure to iterate over db records to perform queries
        cur = elephantsql_client.cursor()

        # Execute commands in order
        cur.execute(command)

        # Close communication with the PostgreSQL database server
        cur.close()

        # Commit the changes
        elephantsql_client.commit()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

def single_insert(elephantsql_client, insert_req):
    """ Execute a single INSERT request """
    cur = elephantsql_client.cursor()
    try:
        cur.execute(insert_req)
        elephantsql_client.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        elephantsql_client.rollback()
        cur.close()
        return 1
    cur.close()

In [70]:
# Creating the connection to database
elephantsql_client = connect(DATABASE, USERNAME, PASSWORD, HOST)

# Create table
create_table(elephantsql_client)

# Inserting data from dataframe --- obviously not optimal but good for demonstration
for index, row in df.iterrows():
    query = "INSERT INTO road_data_table (id, scene, weather, time_of_day, shadows, road_type, traffic, line_condition, lane_count, pavement, detection_preformence) VALUES ('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}')".format(index, row.scene, row.weather, row.time_of_day, row.shadows, row.road_type, row.traffic, row.line_condition, row.lane_count, row.pavement, row.detection_preformence)
    single_insert(elephantsql_client, query)

# Close the connection
elephantsql_client.close()
print('Connection is closed.')

Connecting to the PostgreSQL database...
Connection is closed.


### Retreving road data from postgresql database

In [71]:
# Creating the connection to database
elephantsql_client = connect(DATABASE, USERNAME, PASSWORD, HOST)

# A "cursor", a structure to iterate over db records to perform queries
cur = elephantsql_client.cursor()

command = '''
SELECT scene FROM road_data_table
'''

# Execute commands in order
cur.execute(command)

scene_names = []
scene_list = cur.fetchall()
for tup in scene_list:
    scene_names.append({'label': tup[0], 'value': tup[0]})
print(scene_names)
# Close communication with the PostgreSQL database server
cur.close()

# Commit the changes
elephantsql_client.commit()

# Close the connection
elephantsql_client.close()
print('Connection is closed.')

Connecting to the PostgreSQL database...
[{'label': 'scene_1.mp4', 'value': 'scene_1.mp4'}, {'label': 'scene_2.mp4', 'value': 'scene_2.mp4'}, {'label': 'scene_3.mp4', 'value': 'scene_3.mp4'}, {'label': 'scene_4.mp4', 'value': 'scene_4.mp4'}, {'label': 'scene_5.mp4', 'value': 'scene_5.mp4'}, {'label': 'scene_6.mp4', 'value': 'scene_6.mp4'}, {'label': 'scene_7.mp4', 'value': 'scene_7.mp4'}, {'label': 'scene_8.mp4', 'value': 'scene_8.mp4'}, {'label': 'scene_9.mp4', 'value': 'scene_9.mp4'}, {'label': 'scene_10.mp4', 'value': 'scene_10.mp4'}, {'label': 'scene_11.mp4', 'value': 'scene_11.mp4'}, {'label': 'scene_12.mp4', 'value': 'scene_12.mp4'}, {'label': 'scene_13.mp4', 'value': 'scene_13.mp4'}, {'label': 'scene_14.mp4', 'value': 'scene_14.mp4'}, {'label': 'scene_15.mp4', 'value': 'scene_15.mp4'}, {'label': 'scene_16.mp4', 'value': 'scene_16.mp4'}, {'label': 'scene_17.mp4', 'value': 'scene_17.mp4'}, {'label': 'scene_18.mp4', 'value': 'scene_18.mp4'}]
Connection is closed.
